# Tests comparing models using capymoa

## Installation:
See https://capymoa.org/


```
# CapyMOA requires Java. This checks if you have it installed
java -version

# Install java on ubuntu with sudo apt install openjdk-default

# CapyMOA requires PyTorch. This installs the CPU version
pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

# Install CapyMOA and its dependencies
pip install capymoa

# Check that the install worked
python -c "import capymoa; print(capymoa.__version__)"
```

Import Modules

In [ ]:
from capymoa.regressor import KNNRegressor, AdaptiveRandomForestRegressor
from capymoa.stream._stream import NumpyStream
from capymoa.evaluation import prequential_evaluation
from capymoa.evaluation.visualization import plot_windowed_results
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint  # Import pprint module

Load the Data:




In [ ]:
path_to_csv = Path("/home/bd65/Downloads/power_plant/MHPA-docs-data.csv")
x_features = pd.read_csv(path_to_csv).fillna(0).to_numpy()

There are the following columns:
```
0       1       2        3                4                     5           6           7   
TmStamp,RecNum,batt_volt,mean_wind_speed,mean_wind_direction,std_wind_dir,Max_Gust_Min,Max_Gust_Hr,

8        9              10            11       12     13        14              15
Rain_mm,Barametric_Avg,Air_Temp_Avg,RH_Avg,SlrW_Avg,SlrMJ_Tot,in_bytes_str,Dew_Point_Avg
```


- `TmStamp` is a timestamp, and may be helpful for training (if the model can learn the seasons from it or something)
- `RecNum` is basically the same as timestamp, but it gets reset to zero sometimes
- `SlrW_Avg` is the column we are trying to predict
- `SlrMJ_Tot` is the same as SlrW_Avg, so it needs to be removed from the data
- `in_bytes_str` seems to have some metadata, it's a string e.g `"Q,116,002.68,M,00,_x0003_14"` . For now we can try by just removing it



In [ ]:
# Seperate out the target variable (what we are trying to predict)
class_index: int = 12,  # slrw_avg
targets = x_features[:, class_index]

# remove slrw_avg, slrmj_tot, recnum, tmstamp from x_features, so they are not used as features

x_features = np.delete(x_features, -2, axis=1) # remove in_bytes_str
x_features = np.delete(x_features, class_index, axis=1) # remove slrw_avg
x_features = np.delete(x_features, class_index, axis=1) # remove slrmj_tot
x_features = np.delete(x_features, 0, axis=1) # remove recnum
x_features = np.delete(x_features, 0, axis=1) # remove tmstamp

Now we can create a CapyMoa stream from the data

In [ ]:
stream = NumpyStream(
    x_features,
    targets,
    target_type=target_type,
    feature_names=["batt_volt", "mean_wind_speed", "mean_wind_direction", "std_wind_dir", "Max_Gust_Min", "Max_Gust_Hr","Rain_mm","Barametric_Avg","Air_Temp_Avg","RH_Avg","Dew_Point_Avg"],
)

Create some learner models to test

In [ ]:
ARF_learner = AdaptiveRandomForestRegressor(schema=stream.get_schema(), ensemble_size=10)
knnreg = KNNRegressor(schema=stream.get_schema(), k=3, window_size=1000)

Evaluate with prequential evaluation

In [ ]:
results_arf = prequential_evaluation(stream=stream, learner=ARF_learner, window_size=5000)

Print the results

In [ ]:
pprint(results_arf['windowed'].metrics_per_window())
pprint(results_arf)